In [1]:
import astropy.units as u
import lsdb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import dask
import scipy
import astropy.units as u
import warnings

from dask.distributed import Client
from nested_pandas.utils import count_nested
from dask.distributed import print as dask_print
from lsdb.core.search.pixel_search import PixelSearch
from lsdb.core.search.order_search import OrderSearch
from lsdb.core.search import ConeSearch
from io import StringIO
from nested_pandas import NestedDtype
from pathlib import Path

These are just the 6 fields of LSST commissioning (I wanted a list of a few cone searches, and this is the first one I thought of).

First, we can just look at fetching GAIA in a (smaller) cone in each field.

Alternatively, you can just fetch the full 1.1TB, following directions on https://data.lsdb.io/#Gaia/Gaia_DR3_(GAIA_SOURCE)

```
wget -r -np -nH --cut-dirs=2 -R "*.html*" https://data.lsdb.io/hats/gaia_dr3/gaia/
```

In [5]:
fields = {
    "ECDFS": (53.13, -28.10),  # Extended Chandra Deep Field South
    "EDFS": (59.10, -48.73),  # Euclid Deep Field South
    "Rubin_SV_38_7": (37.86, 6.98),  # Low Ecliptic Latitude Field
    "Rubin_SV_95_-25": (95.00, -25.00),  # Low Galactic Latitude Field
    "47_Tuc": (6.02, -72.08),  # 47 Tuc Globular Cluster
    "Fornax_dSph": (40.00, -34.45),  # Fornax Dwarf Spheroidal Galaxy
}
# Define the radius for selecting sources
selection_radius_arcsec = 0.5 * 3600  # 1/2-degree radius

In [6]:
gaia_full = lsdb.read_hats("https://data.lsdb.io/hats/gaia_dr3/gaia")

In [8]:
for field_name, field_region in fields.items():
    print("cone for ", field_name)
    cone = ConeSearch(
        ra=field_region[0],
        dec=field_region[1],
        radius_arcsec=selection_radius_arcsec,
    )
    
    ## restrict the catalog to just the cone we're interested in
    ## this DOESN'T change the `gaia_full` object, so we can re-use it.
    ## this operation is also "lazy"
    gaia_partial = gaia_full.search(cone)
    
    ## this is the "eager" call, that will return a pandas dataframe with
    ## just the results of the cone search. what you do with it is up to you!
    cone_results = gaia_partial.compute()
    cone_results.to_parquet(f"gaia_{field_name}.pq")
    print("   has", len(cone_results), "rows")
    

cone for  ECDFS
   has 2795 rows
cone for  EDFS
   has 3249 rows
cone for  Rubin_SV_38_7
   has 2522 rows
cone for  Rubin_SV_95_-25
   has 15620 rows
cone for  47_Tuc
   has 157203 rows
cone for  Fornax_dSph
   has 47906 rows


In [11]:
!du -ch gaia_*

30M	gaia_47_Tuc.pq
596K	gaia_ECDFS.pq
704K	gaia_EDFS.pq
8.9M	gaia_Fornax_dSph.pq
548K	gaia_Rubin_SV_38_7.pq
3.3M	gaia_Rubin_SV_95_-25.pq
4.0K	gaia_new.py
44M	total


You could repeat that process for all the catalogs that you're interested in that we offer through data.lsdb.io:

* ZTF: https://data.lsdb.io/#ZTF/ZTF_DR22
* eRosita: https://data.lsdb.io/#eRASS1_Main

I'd not heard of EMU or asas-sn before this. If they're public data sets, and have discrete releases, we can potentially HATS-ify them and host them (but that takes around a month for us to get around to it, assuming we can find the data).

If all of your data is in HATS (or in a pandas dataframe that can be HATS-ified (see [this notebook](https://docs.lsdb.io/en/stable/tutorials/pre_executed/ztf_bts-ngc.html#Put-both-catalogs-to-LSDB-and-plan-cross-match))), then you can do everything within LSDB.

In [18]:
%pip install --quiet nway

Note: you may need to restart the kernel to use updated packages.


In [19]:
from lsdb.core.crossmatch.abstract_crossmatch_algorithm import AbstractCrossmatchAlgorithm
import nwaylib.fastskymatch as match
import hats.pixel_math.healpix_shim as hp

class NWAYCrossmatch(AbstractCrossmatchAlgorithm):
    
    @classmethod
    def validate(match_radius):
        """This should really be implemented to check that we've provided valid arguments"""
        if match_radius < 0:
            raise ValueError("match radius has to be positive, you silly goose.")
        pass
    
    
    def perform_crossmatch(
            self,
            match_radius,
        ):
    
        ## Looks like they want astropy Tables. The per-partition data
        ## comes in as pandas dataframes, so let's just convert them to tables.
        tables = [Table.from_pandas(self.left), Table.from_pandas(self.right)]
        
        ## The catalog names are stashed in this member of AbstractCrossmatchAlgorithm
        table_names = self.suffixes
        
        
        fits_formats = None
        
        ## I can't figure out where source_densities go, but since these are 
        ## healpix tiles, you can find the area real-easily.
        
        area_total = (4 * pi * (180 / pi)**2)
        
        left_area = hp.order2pixarea(self.left_order)
        left_density = len(self.left) / left_area * area_total
        right_area = hp.order2pixarea(self.right_order)
        right_density = len(self.right) / right_area * area_total
        
        # Then you call it, and convert it back to a dataframe.    
        results, columns, match_header = match.match_multiple(tables, 
                                                              table_names, 
                                                              match_radius, 
                                                              fits_formats)
        
        ## you're going to need to do more here, but I'm not sure what right now.
        return results.to_pandas()

In [ ]:
## You can reduce your search space in gaiaXerosita by first matching to the clusters you know about.
## This one doesn't specify the crossmatch algorithm because nearest neighbor is "ok enough".
cluster_gaia = gaia_full.crossmatch(cluster_catalog)

## Then crossmatch that area with xrays from erosita.
xray_counterparts = cluster_gaia.crossmatch(erosita_catalog, algorithm=NWAYCrossmatch).compute()